# Section 1

In [ ]:
import pandas as pd
import re
import nltk
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

fileName = "Assassin.csv"
df = pd.read_csv(f"../DS/{fileName}")
df.head()


,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


In [2]:
df = df.drop(columns=['sender', 'receiver','date','subject', 'urls'])

In [3]:
# Remove rows with missing values
df = df.dropna()
df.isnull().sum()

body     0
label    0
dtype: int64

In [4]:
# Remove rows with duplicated values
df = df.drop_duplicates(subset=['body'])
df.duplicated(subset=['body']).sum()

np.int64(0)

In [5]:
def clean_email_body(html_text):
    # 1. Parse HTML and extract plain text
    try:
        soup = BeautifulSoup(html_text, 'html.parser')
        text = soup.get_text()
    except:
        # Handle cases where the body might not be HTML (e.g., just plain text)
        text = html_text

    # 2. Normalize whitespace 
    text = re.sub(r'\s+', ' ', text)

    # 3. Remove URLs, emails, and punctuation
    text = re.sub(r'http\S+', ' ', text)  # Replace URLs with a space
    text = re.sub(r'\S+@\S+', ' ', text) # Replace emails with a space
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) # Replace non-letters with a space

    # 4. Final cleanup: lowercase and strip leading/trailing spaces
    text = text.lower().strip()
    
    return text

df["cleaned_email_body"] = df["body"].apply(clean_email_body)
df = df.drop(columns="body")
df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_26508\3984029393.py:4: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html_text, 'html.parser')


,label,cleaned_email_body
0,0,date wed aug from ch...
1,0,martin a posted tassos papadopoulos the gree...
2,0,man threatens explosion in moscow thursday aug...
3,0,klez the virus that won t die already the mos...
4,0,in adding cream to spaghetti carbonara which ...


In [6]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # needed for lemmatizer
nltk.download('words')    # for english_vocab

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [7]:
# # Explicitly set NLTK's data directory to the one you downloaded to
# nltk_data_dir = r"C:\Users\AppData\Roaming\nltk_data"
# os.environ["NLTK_DATA"] = nltk_data_dir

# # Add custom path to the search location
# nltk.data.path.append(nltk_data_dir)

# # Confirm
# print("NLTK data paths:")
# for p in nltk.data.path:
#     print("  -", p)

In [8]:
print("stopwords loaded:", len(stopwords.words('english')))
print("tokenizer test:", word_tokenize("hello world!"))

stopwords loaded: 198
tokenizer test: ['hello', 'world', '!']


In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text_2(text, min_words=1):
    # Tokenize
    tokens = word_tokenize(text)

    processed_tokens = []
    for word in tokens:
        # Keep alphabetic tokens only, remove stopwords
        if word not in stop_words:
            processed_tokens.append(lemmatizer.lemmatize(word))

    # Optionally skip very short texts
    if len(processed_tokens) < min_words:
        return None

    return " ".join(processed_tokens)

df['final_text'] = df['cleaned_email_body'].apply(preprocess_text_2)

In [10]:
df = df.drop(columns="cleaned_email_body")
df = df.dropna()
print("Missing values per column:")
df.isna().sum()

Missing values per column:


label         0
final_text    0
dtype: int64

In [11]:
df.head()

,label,final_text
0,0,date wed aug chris garrigues message id reprod...
1,0,martin posted tasso papadopoulos greek sculpto...
2,0,man threatens explosion moscow thursday august...
3,0,klez virus die already prolific virus ever kle...
4,0,adding cream spaghetti carbonara effect pasta ...


In [12]:
# Changing column names
df.rename(columns={'label': 'is_spam'}, inplace=True)

# Reordering columns
df = df[['final_text', 'is_spam']]

# Dropping indicies
df = df.reset_index(drop=True)

df.head()

,final_text,is_spam
0,date wed aug chris garrigues message id reprod...,0
1,martin posted tasso papadopoulos greek sculpto...,0
2,man threatens explosion moscow thursday august...,0
3,klez virus die already prolific virus ever kle...,0
4,adding cream spaghetti carbonara effect pasta ...,0


In [ ]:
def get_filename_without_ext(filename):
  """
  Gets the filename without its extension.
  """
  return os.path.splitext(filename)[0]

filename_without_ext = get_filename_without_ext(fileName)

df.to_csv(f"../cleaned_DS/{filename_without_ext}_cleaned.csv", index=False)